In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd /content/gdrive/MyDrive/PCB_QM/object_detection

Mounted at /content/gdrive
/content/gdrive/MyDrive/PCB_QM/object_detection


In [3]:
# yolo v8를 다운 받기 위해
!pip install ultralytics

from ultralytics import YOLO

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.0/800.0 kB 9.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [4]:
best_weight_path = './best.pt'
model = YOLO(model = best_weight_path)

In [10]:
model_names = model.names
model_names

{0: 'missing_hole',
 1: 'mouse_bite',
 2: 'open_circuit',
 3: 'short',
 4: 'spur',
 5: 'spurious_copper'}

In [ ]:
img_path = "/content/gdrive/MyDrive/PCB_QM/PCB_DATASET/rotation/Open_circuit_rotation/04_open_circuit_10.jpg"
detection = model(img_path)[0]
boxinfos = detection.boxes.data.tolist()
img = cv2.imread(img_path,cv2.IMREAD_COLOR)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
threshold = 0.4
for data in boxinfos :
    x1,y1,x2,y2 = map(int,data[:4])
    confidence_score = round(data[4],2)
    classid = int(data[5])
    name = model_names[classid]
    if confidence_score > threshold:
        model_text = f'{name}_{confidence_score}'
        cv2.rectangle(img, (x1,y1),(x2,y2),(0,255,0),3)
        cv2.putText(img,
                    text= model_text,
                    org = (x1-500, y1 - 5),
                    fontFace = cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale  = 3,
                    color = (255,0,0),
                    thickness = 5)
plt.imshow(img)


In [5]:
import cv2
import matplotlib.pyplot as plt


In [16]:

def img_obj_plot(path,threshold = 0.3):
    detection = model(img_path)[0]
    boxinfos = detection.boxes.data.tolist()
    img = cv2.imread(img_path,cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

    for data in boxinfos :
        x1,y1,x2,y2 = map(int,data[:4])
        confidence_score = round(data[4],2)
        classid = int(data[5])
        name = model_names[classid]
        if confidence_score > threshold:
            model_text = f'{name}_{confidence_score}'
            cv2.rectangle(img, (x1,y1),(x2,y2),(0,255,0),3)
            cv2.putText(img,
                        text= model_text,
                        org = (x1-250, y1 - 5),
                        fontFace = cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale  = 3,
                        color = (255,0,0),
                        thickness = 5)
    plt.imshow(img)

In [6]:
%cd ../

/content/gdrive/MyDrive/PCB_QM


In [7]:
import pandas as pd

In [8]:
pd_data = pd.read_csv('./PCB_LABELING_PLUS_ROTATION.csv')
pd_data.head(5)

,Unnamed: 0,path,error_label,prod_label
0,0,./PCB_DATASET/PCB_USED/01.JPG,0,1
1,1,./PCB_DATASET/PCB_USED/04.JPG,0,4
2,2,./PCB_DATASET/PCB_USED/05.JPG,0,5
3,3,./PCB_DATASET/PCB_USED/06.JPG,0,6
4,4,./PCB_DATASET/PCB_USED/07.JPG,0,7


In [12]:
num_img = len(pd_data)

In [15]:
img_paths = pd_data['path']
img_paths

0                           ./PCB_DATASET/PCB_USED/01.JPG
1                           ./PCB_DATASET/PCB_USED/04.JPG
2                           ./PCB_DATASET/PCB_USED/05.JPG
3                           ./PCB_DATASET/PCB_USED/06.JPG
4                           ./PCB_DATASET/PCB_USED/07.JPG
                              ...                        
1391    ./PCB_DATASET/rotation/Spurious_copper_rotatio...
1392    ./PCB_DATASET/rotation/Spurious_copper_rotatio...
1393    ./PCB_DATASET/rotation/Spurious_copper_rotatio...
1394    ./PCB_DATASET/rotation/Spurious_copper_rotatio...
1395    ./PCB_DATASET/rotation/Spurious_copper_rotatio...
Name: path, Length: 1396, dtype: object

In [13]:
import random

In [25]:
sample_idx = random.sample(range(num_img), 5)

fig = plt.figure(figsize = (50,50))

for cnt,idx in enumerate(sample_idx):
    plt.subplot(len(sample_idx),1,cnt+1)
    img_path = img_paths[idx]
    file_name = img_path.split('/')[-1]
    print(file_name)
    img_obj_plot(img_path,0.05)
    plt.title(file_name)



Output hidden; open in https://colab.research.google.com to view.